In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import psycopg2

In [ ]:
from google.colab import userdata

Will need a background worker to constantly check updates in the DB

In [ ]:
# Connection parameters
conn = psycopg2.connect(
    dbname= userdata.get('DBNAME'),
    user= userdata.get('USERNAME'),
    password= userdata.get('PASSWORD'),
    host= userdata.get('HOST'),
    port= userdata.get('PORT'),
    )

In [ ]:
myquery = "SELECT * FROM foodproduct;"
products_df = pd.read_sql_query(myquery, conn)

In [ ]:
# Combine ingredients into a single string for each food item
ingredient_corpus = products_df['ingredients']

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the ingredient lists
tfidf_matrix = vectorizer.fit_transform(ingredient_corpus)

# Check the matrix shape
# print("TF-IDF Matrix Shape:", tfidf_matrix.shape)

TF-IDF Matrix Shape: (17, 203)


In [ ]:
# Compute cosine similarity between all food items
cosine_sim = cosine_similarity(tfidf_matrix)

# Display the similarity matrix
similarity_df = pd.DataFrame(cosine_sim, index=products_df['name'], columns=products_df['name'])

In [ ]:
health_rating_to_score = {
    'healthy': 2,
    'neutral': 1,
    'unhealthy': -1
}

In [ ]:
def get_health_scores_from_db(conn):
    query = "SELECT name, rating FROM ingredients;"
    ingredients_df = pd.read_sql_query(query, conn)

    # Exclude unwanted names first
    excluded_items = ['Sugar', 'Corn Syrup', 'Cane Sugar', 'Pure Cane Sugar', 'Coffee Caramelized Sugar', 'Corn']
    filtered_df = ingredients_df[~ingredients_df['name'].isin(excluded_items)]

    # Filter for healthy, neutral, and unhealthy ratings
    filtered_df = filtered_df[
        (filtered_df['rating'] == 'healthy') |
        (filtered_df['rating'] == 'neutral') |
        (filtered_df['rating'] == 'unhealthy')
    ]

    # Map health ratings to scores
    filtered_df['health_score'] = filtered_df['rating'].map(health_rating_to_score)
    return filtered_df.set_index('name')['health_score']


health_scores = get_health_scores_from_db(conn)

<ipython-input-8-48fca1fc3523>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ingredients_df = pd.read_sql_query(query, conn)


In [ ]:
def calculate_health_score(ingredients, health_scores):
    score = 0
    for ingredient in ingredients.split(", "):
        if ingredient not in health_scores:  # Skip if ingredient not found
            return None  # Mark this row for exclusion
        score += health_scores[ingredient]
    return score

# Apply to dataset
products_df['health_score'] = products_df['ingredients'].apply(lambda x: calculate_health_score(x, health_scores))

# Drop rows where health_score is None
products_df = products_df.dropna(subset=['health_score'])

In [ ]:
def recommend_healthier_food(item_name, top_n=3):
    # Get the category of the scanned item
    item_row = products_df[products_df['name'] == item_name].iloc[0]
    print(item_row)
    item_score = item_row['health_score']
    item_category = item_row['category'].lower()
    if pd.isna(item_score):
        item_score = 0

    # Filter for healthier items in the same category
    healthier_items = products_df[(products_df['health_score'] > item_score) & (products_df['category'].str.lower() == item_category)]

    # Compute similarity scores
    sim_scores = similarity_df[item_name].loc[healthier_items['name']]

    # Sort and return top recommendations
    recommended_items = sim_scores.sort_values(ascending=False).iloc[:top_n]
    return recommended_items

# Example Usage
recommendations = recommend_healthier_food("3")
print("Healthier Recommendations:\n", recommendations)